## Get Results from old simulation
To run a new simulation, execute the `run_sim.py` script.
If the command below fails, you probably need to run a new simulation to generate data.

In [ ]:
# Specific imports for cellular_raza
# The package is named cr_autophagy
# We want to reload the package when some of the behind-the scenes python functions change
# This is what the importlib statements are for
import importlib
import cr_pool_model as crp
importlib.reload(crp)

# Imports of general-purpose python libraries
import numpy as np
import matplotlib.pyplot as plt
import os
# from pathlib import Path

output_path = crp.get_last_output_path()
# output_path = Path("out/pool_model/2023-12-07-T01-53-41")
domain, initial_cells, meta_params = crp.get_simulation_settings(output_path)
max_iter = max(crp.get_all_iterations(output_path))

minutes = 60
hours = 24

In [ ]:
iter_0_particles = crp.get_elements_at_iter(output_path, 0)
print("Name".ljust(62), "Type")
print("⎯"*80)
for ty, col in zip(iter_0_particles.dtypes, iter_0_particles.columns):
    print("{}".format(col).ljust(62), "{}".format(ty))

In [ ]:
iter_0_voxels = crp.get_elements_at_iter(output_path, 0, element_path="voxel_storage")
print("Name".ljust(62), "Type")
print("⎯"*80)
for ty, col in zip(iter_0_voxels.dtypes, iter_0_voxels.columns):
    print("{}".format(col).ljust(62), "{}".format(ty))

In [ ]:
all_iterations = crp.get_all_iterations(output_path)

def analyze_voxel_data(output_path, iteration):
    # Calculate total extracellular concentrations
    entry = crp.get_elements_at_iter(output_path, iteration, element_path="voxel_storage")
    mi = np.array([x for x in entry["element.voxel.min"]])
    ma = np.array([x for x in entry["element.voxel.max"]])
    diff = np.abs(ma-mi)
    conc = np.array([c for c in entry["element.voxel.extracellular_concentrations"]])
    volume = np.prod(diff, axis=1)
    conc_total = np.sum(volume[:,None] * conc, axis=0)

    return {
        "iteration": iteration,
        "time": iteration*meta_params.dt/minutes/hours,
        "nutrients_conc_total": conc_total[0],
        "inhib_total": conc_total[1],
    }

def _analyze_voxel_data_helper(args):
    return analyze_voxel_data(*args)

def analyze_cell_data(output_path, iteration):
    entry = crp.get_elements_at_iter(output_path, iteration)

    # Calculate number of cells for species
    bacteria_1 = entry[entry["element.cell.cellular_reactions.species"]=="S1"]
    bacteria_2 = entry[entry["element.cell.cellular_reactions.species"]=="S2"]
    bacteria_count_1 = len(bacteria_1)
    bacteria_count_2 = len(bacteria_2)

    # Calculate the number of bacteria in lag phase
    bacteria_in_lag_phase_1 = len(bacteria_1[bacteria_1["element.cell.cellular_reactions.lag_phase_active"]==True])
    bacteria_in_lag_phase_2 = len(bacteria_2[bacteria_2["element.cell.cellular_reactions.lag_phase_active"]==True])

    # Calculate total intracellular concentrations
    volume = np.pi*np.array([x for x in entry["element.cell.interaction.cell_radius"]])**2
    conc = np.array([x for x in entry["element.cell.cellular_reactions.intracellular_concentrations"]])
    conc_total = np.sum(volume[:,None] * conc, axis=0)

    # Calculate the entropy at this step
    entropy = crp.calculate_entropy(output_path, iteration)

    # Return combined results
    return {
        "iteration": iteration,
        "time": iteration*meta_params.dt/minutes/hours,
        "bacteria_count_1": bacteria_count_1,
        "bacteria_count_2": bacteria_count_2,
        "bacteria_count_total": bacteria_count_1 + bacteria_count_2,
        "bacteria_in_lag_phase_1": bacteria_in_lag_phase_1,
        "bacteria_in_lag_phase_2": bacteria_in_lag_phase_2,
        "bacteria_in_lag_phase_total": bacteria_in_lag_phase_1 + bacteria_in_lag_phase_2,
        "nutrients_conc_total": conc_total[0],
        "inhib_conc_total": conc_total[1],
        "entropy": entropy,
    }

def _analyze_cell_data_helper(args):
    return analyze_cell_data(*args)

pool = crp.mp.Pool(30)

args = [(output_path, iteration) for iteration in crp.get_all_iterations(output_path)]
data_cells = crp.pd.DataFrame(crp.tqdm.tqdm(pool.imap_unordered(_analyze_cell_data_helper, args), total=len(args))).sort_values("iteration").reset_index(drop=True)
data_voxels = crp.pd.DataFrame(crp.tqdm.tqdm(pool.imap_unordered(_analyze_voxel_data_helper, args), total=len(args))).sort_values("iteration").reset_index(drop=True)

pool.close()
pool.join()

In [ ]:
fig, ax1 = plt.subplots()

data_voxels.plot(x="time", y="nutrients_conc_total", ax=ax1, label="Extracellular Nutrients", color='#808080')
ax1.set_ylabel("Nutrient Amount")
ax1.set_xlabel("Time [days]")

ax2 = ax1.twinx()
data_cells.plot(x="time", y="bacteria_count_1", ax=ax2, label="Species 1", color="#252B33")
data_cells.plot(x="time", y="bacteria_count_2", ax=ax2, label="Species 2", color="#D14027")
data_cells.plot(x="time", y="bacteria_count_total", ax=ax2, label="Combined", color="#7E57A5")
ax2.set_ylabel("Bacteria Count")

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
fig.tight_layout()
fig.savefig(f"{output_path}/cell_growth.png")
fig.savefig(f"{output_path}/cell_growth.pdf")
plt.show(fig)

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Cells in Lag-Phase")
data_cells.plot(x="time", y="bacteria_in_lag_phase_1", ax=ax, label="Species 1", color="#252B33")
data_cells.plot(x="time", y="bacteria_in_lag_phase_2", ax=ax, label="Species 2", color="#D14027")
data_cells.plot(x="time", y="bacteria_in_lag_phase_total", ax=ax, label="Combined", color="#7E57A5")
ax.legend()
ax.set_xlabel("Time [days]")
ax.set_ylabel("Bacteria Count")
fig.tight_layout()
fig.savefig(f"{output_path}/lag_phase.png")
fig.savefig(f"{output_path}/lag_phase.pdf")
plt.show(fig)

In [ ]:
entropies = np.array([x for x in data_cells["entropy"]])

fig, ax = plt.subplots()
ax.plot(data_cells["time"], entropies[:,0], label="Species 1", color='#D06062')
ax.plot(data_cells["time"], entropies[:,1], label="Species 2", color='#4E89B1')
ax.legend()
ax.set_xlabel("Time [days]")
ax.set_ylabel("Shannon Entropy")
fig.tight_layout()
fig.savefig(output_path / "entropy.png")
fig.savefig(output_path / "entropy.pdf")
plt.show(fig)

## Generate Screenshots for every Iteration

In [ ]:
def _save_psychic_image(output_path, iteration):
    import scipy as sp
    data = crp.get_elements_at_iter(output_path, iteration)
    domain, _, _ = crp.get_simulation_settings(output_path)
    fig, ax = crp._create_base_canvas(domain)

    data1 = data[data["element.cell.cellular_reactions.species"]=="S1"]
    data2 = data[data["element.cell.cellular_reactions.species"]!="S1"]

    Z1 = crp.calculate_spatial_density(data1, domain, weights=True)
    Z2 = crp.calculate_spatial_density(data2, domain, weights=True)

    Z = Z1/(Z1+Z2)

    ax.imshow(np.rot90(Z), cmap=plt.cm.viridis,
        extent=[0.0, domain.size, 0.0, domain.size])

    df_cells = crp.get_elements_at_iter(output_path, iteration)

    crp._plot_bacteria(df_cells, ax)

    ax.set_xlim([0.0, domain.size])
    ax.set_ylim([0.0, domain.size])

    # ax.set_title(f"Entropy {sp.stats.entropy(Z.reshape(-1)):4.2f}")
    crp._plot_labels(fig, ax, n_bacteria_1=len(data1), n_bacteria_2=len(data2))

    save_path = crp._determine_image_save_path(output_path, iteration)
    fig.savefig(save_path, bbox_inches='tight', pad_inches=0)

def _save_psychic_image_helper(args):
    _save_psychic_image(*args)

# _save_psychic_image(output_path, max_iter)

# import multiprocessing as mp
# import tqdm
# args = [(output_path, iteration) for iteration in crp.get_all_iterations(output_path)]
# _ = list(tqdm.tqdm(mp.Pool().imap_unordered(_save_psychic_image_helper, args), total=len(args)))

In [ ]:
# Save snapshots to local folder
crp.save_all_snapshots(output_path, threads=-1)

# Also create a movie with ffmpeg
bashcmd = f"ffmpeg -v quiet -stats -y -r 30 -f image2 -pattern_type glob -i '{output_path}/images/*.png' -c:v h264 -pix_fmt yuv420p -strict -2 {output_path}/movie_4.mp4"
os.system(bashcmd)